<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [112]:
#!pip3 install folium
#!pip3 install wget

In [113]:
import folium
import wget
import pandas as pd

In [114]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [115]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [116]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [117]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [118]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [119]:
# Assume 'launch_sites_df' is your DataFrame and it has columns 'Lat', 'Long', and 'Launch Site'.
# We will iterate over each row in the DataFrame to plot the launch sites.

# Iterate through each row in the DataFrame using iterrows(), which is a generator
# that yields both the index and row (as a Series).
for index, row in launch_sites_df.iterrows():

    # Create a Circle object on the map using the 'Lat' and 'Long' values from the DataFrame.
    # The 'location' parameter takes a list or tuple of latitude and longitude values.
    # The 'radius' parameter specifies the radius of the circle in meters.
    # The 'color' parameter specifies the color of the circle's outline.
    # The 'fill' parameter determines whether to fill the circle with color.
    # The 'fill_color' parameter specifies the color to fill the circle.
    folium.Circle(
        location=[row['Lat'], row['Long']],  # Latitude and Longitude of the launch site
        radius=500,                          # Radius in meters
        color='blue',                        # Color of the circle's border
        fill=True,                           # Enable fill
        fill_color='blue'                    # Fill color
    ).add_to(site_map)  # Add the Circle object to the site_map

    # Create a Marker object to mark the location of the launch site.
    # The Marker is placed at the same latitude and longitude as the Circle.
    # A popup is also attached to the Marker using the 'popup' parameter.
    # The Popup will show the launch site's name when the Marker is clicked.
    # The 'parse_html' parameter allows for HTML in the popup, providing more flexibility in formatting.
    folium.Marker(
        location=[row['Lat'], row['Long']],       # Latitude and Longitude for the Marker
        popup=folium.Popup(row['Launch Site'],    # Popup text, which is the name of the launch site
                           parse_html=True)       # Allow HTML content within the popup
    ).add_to(site_map)  # Add the Marker object to the site_map

# After all circles and markers are added, display the Folium map in a Jupyter notebook.
# The map will show the added circles and markers, and users can interact with it.
site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [120]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [121]:
marker_cluster = MarkerCluster()


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [122]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

In [123]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [124]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed,
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
# Iterate over each row in spacex_df to create a marker for each launch.
for index, row in spacex_df.iterrows():
    # Create a Folium Marker object with its coordinates and a popup showing the launch site name.
    marker = folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch Site'],
        # Here we use Folium's Icon class to specify the color of the marker.
        # The color is determined by the 'marker_color' value for each row.
        icon=folium.Icon(color=row['marker_color'])
    )
    # Add the marker to the MarkerCluster object.
    marker_cluster.add_child(marker)

# Add the MarkerCluster object to the Folium Map object.
site_map.add_child(marker_cluster)

# Display the map in the Jupyter notebook.
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [125]:
# Define a JavaScript formatter function.
# This function will format the numbers (latitude and longitude values) to a fixed number of decimal places.
# Here, it's set to 5 decimal places with the '.Util.formatNum(num, 5)' function call.
formatter = "function(num) {return L.Util.formatNum(num, 5);};"

# Create a MousePosition object.
# This object is initialized with several options that define how the mouse position will be displayed.
mouse_position = MousePosition(
    position='topright',  # Position of the coordinate display on the map (top right corner).
    separator=' Long: ',  # Separator between the latitude and longitude values.
    empty_string='NaN',   # What to display when there is no position (e.g., when the mouse is not over the map).
    lng_first=False,      # Display latitude before longitude (set to True to reverse).
    num_digits=20,        # Number of digits to be displayed after the decimal point (this is very high and could be reduced).
    prefix='Lat: ',       # Text to be displayed before the latitude value.
    # Formatter functions for latitude and longitude. These functions are called to format
    # the displayed values of latitude and longitude. We're using the same formatter for both here.
    lat_formatter=formatter,
    lng_formatter=formatter
)

# Add the MousePosition object to the site_map as a child layer.
# This enables the coordinate display on the map.
site_map.add_child(mouse_position)

# Display the map in the Jupyter notebook.
site_map


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [126]:
from math import sin, cos, sqrt, atan2, radians

# Define a function to calculate the distance between two points on the Earth.
def calculate_distance(lat1, lon1, lat2, lon2):
    # The approximate radius of the earth in kilometers.
    R = 6373.0

    # Convert latitude and longitude from degrees to radians.
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    # Calculate the differences in coordinates.
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Use the Haversine formula to calculate the distance.
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Calculate the distance using the radius of the Earth.
    distance = R * c

    # Return the distance.
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [127]:
# find coordinate of the closet coastline
launch_site_lat = 28.562302
launch_site_lon = -80.577356
coastline_lat = 28.5636
coastline_lon = -80.56807
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_coastline

0.9185876930343655

_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [128]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property
# for example
# Create a folium.Marker to display the distance
from folium import DivIcon
# Remove any existing markers if this is run multiple times
for child in list(site_map._children):
    if isinstance(site_map._children[child], folium.map.Marker):
        del site_map._children[child]

# Create a folium.Marker to display the distance at the coastline point
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(75,18),
        html='<div style="font-size: 16pt; color: #d35400;"><b>{:.2f} km</b></div>'.format(distance_coastline),
    )
)

# Add the custom marker to the map
site_map.add_child(distance_marker)
site_map

_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [129]:
# Coordinates for the launch site and the nearest coastline point
launch_site_coord = [28.562302, -80.577356]
coastline_coord = [28.5636, -80.56807]
# Create a list of points to form the line, each point being a list of latitude and longitude
line_points = [launch_site_coord, coastline_coord]

# Create a folium.PolyLine object with the defined points, set the weight and color
lines = folium.PolyLine(locations=line_points, weight=2, color='blue')

# Add the PolyLine to the map
site_map.add_child(lines)

# Display the map
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
